In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai.vision import *
from pathlib import Path
from fastai.callbacks.mixup import MixUpCallback
from fastai.callbacks.oversampling import OverSamplingCallback

In [3]:
path = Path('/home/rubn/Desktop/og')

In [4]:
dx = ImageList.from_folder(path).split_by_folder(train='train', valid='val').label_from_folder().databunch().normalize(imagenet_stats)

In [5]:
dx

ImageDataBunch;

Train: LabelList (5511 items)
x: ImageList
Image (3, 300, 1000),Image (3, 300, 1000),Image (3, 300, 1000),Image (3, 300, 1000),Image (3, 300, 1000)
y: CategoryList
Healthy,Healthy,Healthy,Healthy,Healthy
Path: /home/rubn/Desktop/og;

Valid: LabelList (1381 items)
x: ImageList
Image (3, 300, 1000),Image (3, 300, 1000),Image (3, 300, 1000),Image (3, 300, 1000),Image (3, 300, 1000)
y: CategoryList
Healthy,Healthy,Healthy,Healthy,Healthy
Path: /home/rubn/Desktop/og;

Test: None

In [6]:
learn = cnn_learner(dx, models.vgg19_bn, callback_fns=[MixUpCallback, OverSamplingCallback]).load('/home/rubn/Desktop/og/models/tmp')

In [ ]:
#learn = load_learner('/home/rubn/FACO-WORKS/Disease Model/app', 'covid_vgg19_overmixup.pkl')

In [7]:
learn.data.batch_size = 8

In [ ]:
learn.lr_find()
learn.recorder.plot(skip_start=0, skip_end=2, suggestion=True)

In [8]:
lr = 1e-06

In [ ]:
learn.fit_one_cycle(4, slice(lr))

In [ ]:
learn.unfreeze()
learn.freeze_to(3)
learn.fit_one_cycle(2, max_lr=slice(lr/10, lr))  

epoch,train_loss,valid_loss,time


In [ ]:
preds,y,losses = learn.get_preds(with_loss=True)
interp = ClassificationInterpretation(learn, preds, y, losses)

In [ ]:
interp.plot_confusion_matrix(normalize=True)

In [ ]:
learn.export(path/'covid_vgg19_overmixup.pkl')